In [2]:
import os
import time
import csv
import signal
import random
import itertools
import numpy as np
from concurrent.futures import ProcessPoolExecutor, TimeoutError

from chocoq.problems.facility_location_problem import generate_flp
from chocoq.problems.graph_coloring_problem import generate_gcp
from chocoq.problems.k_partition_problem import generate_kpp
from chocoq.problems.job_scheduling_problem import generate_jsp
from chocoq.problems.traveling_salesman_problem import generate_tsp
from chocoq.problems.set_cover_problem import generate_scp
from chocoq.solvers.optimizers import CobylaOptimizer, AdamOptimizer
from chocoq.solvers.qiskit import (
    PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver, 
    AerGpuProvider, AerProvider, FakeBrisbaneProvider, FakeKyivProvider, FakeTorinoProvider, DdsimProvider,
)

import pandas as pd
pd.set_option('display.max_rows', None)  # display all rows
pd.set_option('display.max_columns', None)  # display all columns.


## Configure the problems

In [ ]:
num_cases = 10  # The number of cases in each benchmark
problem_scale = 1 # The problem scale, 1 is the minimal scale like F1,K1,G1 in Table 1 of paper
#2 means F2 K2 ... In CPU version, this benchmarks with higher scale is much slower when solving with baselines.

### generate the problem to be solved

In [3]:
flp_problems_pkg, flp_configs_pkg = generate_flp(num_cases, [(1, 2), (2, 3), (3, 3), (3, 4)][:problem_scale], 1, 20)
gcp_problems_pkg, gcp_configs_pkg = generate_gcp(num_cases, [(3, 1), (3, 2), (4, 1), (4, 2)][:problem_scale])
kpp_problems_pkg, kpp_configs_pkg = generate_kpp(num_cases, [(4, 2, 3), (5, 3, 4), (6, 3, 5), (7, 3, 6)][:problem_scale], 1, 20)
jsp_problems_pkg, jsp_configs_pkg = generate_jsp(num_cases, [(2, 2, 3), (2, 3, 4), (3, 3, 5), (3, 4, 6)][:problem_scale], 1, 20)
scp_problems_pkg, scp_configs_pkg = generate_scp(num_cases, [(4, 4), (5, 5), (6, 6), (7, 7)][:problem_scale])

configs_pkg = flp_configs_pkg + gcp_configs_pkg + kpp_configs_pkg + jsp_configs_pkg + scp_configs_pkg
with open(f"1_table.config", "w") as file:
    for pkid, configs in enumerate(configs_pkg):
        for problem in configs:
            file.write(f'{pkid}: {problem}\n')


## Evaluation on circuit depth

In [ ]:
new_path = '1_table_depth'

problems_pkg = flp_problems_pkg + gcp_problems_pkg + kpp_problems_pkg + jsp_problems_pkg + scp_problems_pkg

metrics_lst = ['culled_depth', 'num_params']
solvers = [PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver]
headers = ["pkid", 'method', 'layers'] + metrics_lst

def process_layer(prb, num_layers, solver, metrics_lst):
    opt = CobylaOptimizer(max_iter=200)
    ddsim = DdsimProvider()
    cpu = AerProvider()
    gpu = AerGpuProvider()
    used_solver = solver(
        prb_model = prb,
        optimizer = opt,
        # Select CPU or GPU simulator
        # cpu simulator, comment it when use GPU
        provider = cpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        # uncomment the line below to use GPU simulator
        # provider = gpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        num_layers = num_layers,
        shots = 1024,
    )
    metrics = used_solver.circuit_analyze(metrics_lst)
    return metrics

if __name__ == '__main__':
    set_timeout = 60 * 60 * 24 # Set timeout duration
    num_complete = 0
    with open(f'{new_path}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write headers once

    num_processes_cpu = os.cpu_count() // 2
    with ProcessPoolExecutor(max_workers=num_processes_cpu) as executor:
        futures = []
        for solver in solvers:
            for pkid, problems in enumerate(problems_pkg):
                for problem in problems:
                    if solver == ChocoSolver:
                        num_layers = 1
                    else:
                        num_layers = 5
                    future = executor.submit(process_layer, problem, num_layers, solver, metrics_lst)
                    futures.append((future, pkid, solver.__name__, num_layers))

        start_time = time.perf_counter()
        for future, pkid, solver, num_layers in futures:
            current_time = time.perf_counter()
            remaining_time = max(set_timeout - (current_time - start_time), 0)
            diff = []
            try:
                result = future.result(timeout=remaining_time)
                diff.extend(result)
                print(f"Task for problem {pkid}, num_layers {num_layers} executed successfully.")
            except MemoryError:
                diff.append('memory_error')
                print(f"Task for problem {pkid}, num_layers {num_layers} encountered a MemoryError.")
            except TimeoutError:
                diff.append('timeout')
                print(f"Task for problem {pkid}, num_layers {num_layers} timed out.")
            finally:
                row = [pkid, solver, num_layers] + diff
                with open(f'{new_path}.csv', mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(row)  # Write row immediately
                num_complete += 1
                if num_complete == len(futures):
                    print(f'Data has been written to {new_path}.csv')
                    for process in executor._processes.values():
                        os.kill(process.pid, signal.SIGTERM)

Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license -

In [5]:
file_path = '1_table_depth.csv'

df = pd.read_csv(file_path)

grouped_df = df.groupby(['pkid', 'layers', 'method'], as_index=False).agg({
    "culled_depth": 'mean',
})

values = ["culled_depth"]
pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=values)

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([values, method_order]))

pivot_df


culled_depth                                   
     PenaltySolver CyclicSolver HeaSolver ChocoSolver
pkid                                                 
0             40.0         65.0      30.0        44.0
1            135.2        200.0      60.0       167.3
2             86.6        144.0      40.0       115.1
3             52.0        115.0      35.0        90.0
4            126.4        126.4      45.0       109.2

## Evaluation on other metrics

In [6]:
new_path = '1_table_evaluate'

problems_pkg = list(
    itertools.chain(
        enumerate(flp_problems_pkg),
        enumerate(gcp_problems_pkg),
        enumerate(kpp_problems_pkg),
        enumerate(jsp_problems_pkg),
        enumerate(scp_problems_pkg),
    )
)

solvers = [PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver]
evaluation_metrics = ['best_solution_probs', 'in_constraints_probs', 'ARG', 'iteration_count', 'classcial', 'quantum', 'run_times']
headers = ['pkid', 'pbid', 'layers', "variables", 'constraints', 'method'] + evaluation_metrics

def process_layer(prb, num_layers, solver):
    opt = CobylaOptimizer(max_iter=200)
    ddsim = DdsimProvider()
    cpu = AerProvider()
    gpu = AerGpuProvider()
    prb.set_penalty_lambda(400)
    used_solver = solver(
        prb_model = prb,
        optimizer = opt,
        # 根据配置的环境选择CPU或GPU
        provider = cpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        # provider = gpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        num_layers = num_layers,
        shots = 1024,
    )
    used_solver.solve()
    eval = used_solver.evaluation()
    time = list(used_solver.time_analyze())
    run_times = used_solver.run_counts()
    return eval + time + [run_times]

if __name__ == '__main__':
    all_start_time = time.perf_counter()
    set_timeout = 60 * 60 * 2 # Set timeout duration
    num_complete = 0

    with open(f'{new_path}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # 写入标题

    num_processes_cpu = os.cpu_count()
    # pkid-pbid: 问题包序-包内序号
    for pkid, (diff_level, problems) in enumerate(problems_pkg):
        for solver in solvers:
            if solver in [PenaltySolver, CyclicSolver, HeaSolver]:
                num_processes = 2**(4 - diff_level) + 1
            else:
                num_processes = 100

            with ProcessPoolExecutor(max_workers=num_processes) as executor:
                futures = []
                if solver == ChocoSolver:
                    layer = 1
                else:
                    layer = 5

                for pbid, prb in enumerate(problems):
                    print(f'{pkid}-{pbid}, {layer}, {solver} build')
                    future = executor.submit(process_layer, prb, layer, solver)
                    futures.append((future, prb, pkid, pbid, layer, solver.__name__))

                start_time = time.perf_counter()
                for future, prb, pkid, pbid, layer, solver in futures:
                    current_time = time.perf_counter()
                    remaining_time = max(set_timeout - (current_time - start_time), 0)
                    diff = []
                    try:
                        metrics = future.result(timeout=remaining_time)
                        diff.extend(metrics)
                        print(f"Task for problem {pkid}-{pbid} L={layer} {solver} executed successfully.")
                    except MemoryError:
                        print(f"Task for problem {pkid}-{pbid} L={layer} {solver} encountered a MemoryError.")
                        for dict_term in evaluation_metrics:
                            diff.append('memory_error')
                    except TimeoutError:
                        print(f"Task for problem {pkid}-{pbid} L={layer} {solver} timed out.")
                        for dict_term in evaluation_metrics:
                            diff.append('timeout')
                    except Exception as e:
                        print(f"An error occurred: {e}")
                    finally:
                        row = [pkid, pbid, layer, len(prb.variables), len(prb.lin_constr_mtx), solver] + diff
                        with open(f'{new_path}.csv', mode='a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(row)  # Write row immediately
                        num_complete += 1
                        if num_complete == len(futures):
                            print(f'problem_pkg_{pkid} has finished')
                            for process in executor._processes.values():
                                os.kill(process.pid, signal.SIGTERM)
    print(f'Data has been written to {new_path}.csv')
    print(time.perf_counter()- all_start_time)

0-0, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-1, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build


0-2, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-3, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-4, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-5, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
Restricted license - for non-production use only - expires 2025-11-24
0-6, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-7, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-8, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
0-9, 5, <class 'chocoq.solvers.qiskit.penalty.PenaltySolver'> build
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-product

In [3]:
df = pd.read_csv('1_table_evaluate.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid', 'variables', 'constraints'], columns='method', values=["best_solution_probs",'in_constraints_probs', 'ARG'])
method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([["best_solution_probs",'in_constraints_probs', 'ARG'], method_order]))

pivot_df 

best_solution_probs                         \
                                 PenaltySolver CyclicSolver HeaSolver   
pkid variables constraints                                              
0    6         3                      3.378906    11.074219  1.132812   
1    12        6                      0.546875    23.515625  0.195312   
2    8         6                      1.367188     8.384487  1.618304   
3    7         4                      2.783203    12.148438  3.603516   
4    9         4                      2.646484     8.164062  1.337891   

                                       in_constraints_probs               \
                           ChocoSolver        PenaltySolver CyclicSolver   
pkid variables constraints                                                 
0    6         3             54.501953            14.082031    39.277344   
1    12        6             27.216797             3.085938    59.208984   
2    8         6             79.516602             3.431920    28.585379   
3    7         4             71.044922             9.169922    40.556641   
4    9         4             47.392578             7.177734    18.154297   

                                                            ARG               \
                            HeaSolver ChocoSolver PenaltySolver CyclicSolver   
pkid variables constraints                                                     
0    6         3            10.849609       100.0     54.519551    32.661361   
1    12        6             0.566406       100.0     87.005576    25.532021   
2    8         6             5.161830       100.0    399.140673   174.964849   
3    7         4            12.294922       100.0     62.946534    28.960798   
4    9         4             4.619141       100.0    672.766305   380.494383   

                                                    
                             HeaSolver ChocoSolver  
pkid variables constraints                          
0    6         3             62.456379    0.738041  
1    12        6            140.011265    0.429326  
2    8         6            417.037316    0.160653  
3    7         4             64.143603    0.071310  
4    9         4            752.124557    0.264990

### The method we use to calculate the improvement is as follows: for each benchmark, we compute the improvement ratio. Then, we take the arithmetic mean of the improvement ratios across all benchmarks.

Since all the experiments are run at scale 1 (the simplest problem), the performance of other baselines is also quite good. As a result, the improvement ratios appear to be smaller compared to those in Table 1 of the paper.

In [5]:
# Row-wise ratio calculation
methods = ['PenaltySolver', 'CyclicSolver', 'HeaSolver']
ratios_rowwise = []

for idx, row in pivot_df.iterrows():
    choco_best_solution_probs = row['best_solution_probs', 'ChocoSolver']
    choco_in_constraints_probs = row['in_constraints_probs', 'ChocoSolver']
    choco_ARG = row['ARG', 'ChocoSolver']

    for method in methods:
        ratio_best_solution_probs = choco_best_solution_probs / row['best_solution_probs', method]
        ratio_in_constraints_probs = choco_in_constraints_probs / row['in_constraints_probs', method]
        ratio_ARG = row['ARG', method] / choco_ARG
        
        ratios_rowwise.append({
            'pkid': row.name[0], 
            'variables': row.name[1], 
            'constraints': row.name[2],
            'method': method,
            'ratio_best_solution_probs': ratio_best_solution_probs,
            'ratio_in_constraints_probs': ratio_in_constraints_probs,
            'ratio_ARG': ratio_ARG
        })

# Convert the result into a DataFrame
ratios_rowwise_df = pd.DataFrame(ratios_rowwise)

# Calculate the average ratio for each metric
ratios_avg_df = ratios_rowwise_df.groupby('method').mean()[['ratio_best_solution_probs', 'ratio_in_constraints_probs', 'ratio_ARG']]
ratios_avg_df


,ratio_best_solution_probs,ratio_in_constraints_probs,ratio_ARG
method,,,
CyclicSolver,5.443156,3.141449,606.963853
HeaSolver,58.347330,46.984821,1348.890895
PenaltySolver,33.498539,18.696343,1236.515061
